In [ ]:
from netgen.occ import *
from ngsolve import *
from ngsolve.webgui import Draw
import numpy as np
from newtonmethod import NewtonWithLinesearch
import matplotlib.pyplot as plt


height = 50
thickness = 0.5
radius = 30
radius_in = 3
radius_top = 100
m_top = height + np.sqrt(radius_top**2-radius**2)

E = 1
nu = 0.5

mu = E/2/(1+nu)

pressure = 0.0001


cyl = Cylinder (Pnt(0,0,thickness/2), Z, r=radius, h=height+3*thickness, mantle="mantle")

sp = Sphere( (radius_in*0,0,m_top), radius_top)
sp.faces.name="sphere"
sp.faces.maxh=7
sp.faces.col=(0,0,1)

sol = cyl-sp
for (i,f) in enumerate(sol.faces): f.name = f"face{i}"
tank = Glue( sol.faces )

hole = Cylinder(Pnt(0,0,-thickness), Z, r=radius_in, h=2*thickness )
tank = tank - hole

tank.edges.Nearest((0.1,0,0)).name = "holeedge"

tank.edges[4].hpref = 1
tank.edges[8].hpref = 1

# Draw (tank)

In [ ]:
halftank = tank - Box(Pnt(0,-2*radius,-height), Pnt(2*radius,2*radius,2*height))
halftank.edges[5].name = "sym"
halftank.edges[1].name = "sym"
halftank.edges[3].name = "sym"
halftank.edges[7].name = "sym"
halftank.edges[8].name = "sym"
halftank.edges[10].name = "sym"

Draw(halftank)

ngmesh = OCCGeometry(halftank).GenerateMesh(maxh=10, perfstepsend=5)
order = 3
mesh = Mesh(ngmesh)
mesh.RefineHP(levels=1, factor=0.2)
mesh.Curve(order)

Draw (mesh)

In [ ]:
fes_mom = HDivDivSurface(mesh, order=order-1, discontinuous=True)
fes_u = VectorH1(mesh, order=order, dirichlet_bbnd="holeedge", dirichletx_bbnd="sym")
fes_hyb = HDivSurface(mesh, order=order-1, orderinner=0, dirichlet_bbnd="holeedge|sym")
fes_curlcurl = HCurlCurl(mesh, order=order, discontinuous=True, print=True)
fes_num = FESpace("number", mesh)


fes  = fes_u*fes_hyb*fes_curlcurl*fes_curlcurl*fes_curlcurl*fes_mom*fes_num



q = GridFunction(fes, name="solution")
u = q.components[0]
eps = q.components[2]
kappa = q.components[4]
mom = q.components[5]
pres = q.components[6]


Nsurf = specialcf.normal(mesh.dim) # surface normal N
t     = specialcf.tangential(mesh.dim)
nel   = Cross(Nsurf, t) # in-plane edge normal

gradN = specialcf.Weingarten(3) 


A = Id(mesh.dim) - OuterProduct(Nsurf,Nsurf)

cfnphys = Normalize(Cof(A+Grad(u))*Nsurf)

loadpar = Parameter(0)
bias = Parameter(0)

In [ ]:
fesVF = VectorFacetSurface(mesh, order=order)
averednv = GridFunction(fesVF)
averednv_start = GridFunction(fesVF)


fesF = FacetSurface(mesh, order=0)
gfclamped = GridFunction(fesF)
gfclamped.Set(1,definedon=mesh.BBoundaries("holeedge|sym"))

n_ = fesVF.TrialFunction()
n_.Reshape((3,))
bfF = BilinearForm(fesVF, symmetric=True)
bfF += Variation( (0.5*n_*n_ - ((1-gfclamped)*cfnphys+gfclamped*Nsurf)*n_)*ds(element_boundary=True))


def ComputeAveredNV(averednv):
    rf = averednv.vec.CreateVector()
    bfF.Apply(averednv.vec, rf)
    bfF.AssembleLinearization(averednv.vec)
    invF = bfF.mat.Inverse(fesVF.FreeDofs(), inverse="sparsecholesky")
    averednv.vec.data -= invF*rf


ComputeAveredNV(averednv)
ComputeAveredNV(averednv_start)

cfn  = Normalize(CoefficientFunction( averednv.components ))
cfnR = Normalize(CoefficientFunction( averednv_start.components ))

B0 = GridFunction(fes_curlcurl)
def ComputeB(B0):
    # for precomputing B0
    fes3B0 = HDivSurface(mesh, order=order-1, orderinner=0, dirichlet_bbnd="holeedge")
    fes_B0  = fes3B0*fes_curlcurl*fes_mom
    sol = GridFunction(fes_B0)

    hyb_, B_, mom_ = fes_B0.TrialFunction()
    hyb_, B_, mom_ = hyb_.Trace(), B_.Trace(), mom_.Trace()

    bfA0 = BilinearForm(fes_B0, symmetric=True)
    bfA0 += Variation( InnerProduct(B_, B_)*ds).Compile()
    bfA0 += Variation( (-InnerProduct(mom_, B_ + gradN))*ds ).Compile()
    bfA0 += Variation( (acos(nel*cfnR)-np.pi/2-hyb_*nel)*(mom_*nel)*nel*ds(element_boundary=True ) ).Compile()

    sol.vec[:] = 0
    NewtonWithLinesearch(bfA0, sol.vec)
        
    B0.vec.data = sol.components[1].vec

ComputeB(B0)
Draw(B0, mesh, "B")


In [ ]:
u_, hyb_, eps_, R_, kappa_, mom_, pres_ = fes.TrialFunction()
mom_, hyb_, eps_, R_, kappa_ = mom_.Trace(), hyb_.Trace(), eps_.Trace(), R_.Operator("dualbnd"), kappa_.Trace()

Fsurf_    = grad(u_).Trace() + A
Csurf_    = Fsurf_.trans*Fsurf_
epssurf_ = 0.5*(Csurf_ - A)

nphys_scaled   = (Cof(Fsurf_)*Nsurf) #  normal of deformed surface, including area element
nphys   = Normalize(Cof(Fsurf_)*Nsurf) #  normal of deformed surface
tphys   = Normalize(Fsurf_*t)
nelphys = Cross(nphys,tphys) # in-plane edge normal of deformed surface

Hn_ = CoefficientFunction( (u_.Operator("hesseboundary").trans*nphys), dims=(3,3) )
pnaverage = Normalize( cfn - (tphys*cfn)*tphys )


bfA = BilinearForm(fes, symmetric=True, condense=True)
gausspoints = [(-np.sqrt(3/5), 5/9), (0,8/9), ( np.sqrt(3/5), 5/9)]
for (zi, wi) in gausspoints:
    C_ = 2*(eps_ + thickness*(1)/2*zi*kappa_) + A
    
    I_C = Trace(C_)
    III_C = (Cof(C_)*Nsurf)*Nsurf # C[0]*C[4] - C[1]*C[3] # Det(C)
    I_barC = I_C + 1/III_C
    NeoHooke = 0.5*mu*(I_C + 1/III_C - 3)
    bfA += Variation(thickness*(1)/2*wi*(NeoHooke)*ds).Compile()


bfA += Variation( (InnerProduct(mom_, kappa_ + Hn_ + (1-nphys*Nsurf)*gradN))*ds ).Compile()
bfA += Variation( InnerProduct(eps_-epssurf_, R_)*ds(element_vb=BND) )
bfA += Variation( InnerProduct(eps_-epssurf_, R_)*ds(element_vb=VOL) )
bfA += Variation( -(acos(nel*cfnR)-acos(nelphys*pnaverage)-hyb_*nel)*(mom_*nel)*nel*ds(element_boundary=True) ).Compile()

bfA += Variation((-pres_*(InnerProduct(u_, nphys_scaled)-loadpar))*ds).Compile()


In [ ]:
uinit = GridFunction(fes_u)
uinit.Set(-0.01*CoefficientFunction((0,0,x**2 + y**2 - radius**2)), definedon=mesh.Boundaries("face1"))
Draw(uinit, mesh, "uini", deformation=uinit)

In [ ]:
scenep = Draw(Norm(kappa), mesh, deformation=u)


In [ ]:
loadsteps1 = np.linspace(0,1,41,endpoint=True)
deflist1 = []
preslist1 = []
q.vec[:] = 0
q0 = q.vec.CreateVector()
for l in loadsteps1[:]:
    loadpar.Set(l)
    print(f"loadfactor l = {l}")

    with TaskManager():
        err, nit = NewtonWithLinesearch(bfA, q.vec, maxnewton=20, factor=1.2,scene=scenep)
    if err > 0.05:
        u.vec.data += uinit.vec
        err, nit = NewtonWithLinesearch(bfA, q.vec, maxnewton=50, factor=1.2,scene=scenep, damp=0.5)




    up = u(mesh(0,0,m_top-radius_top,BBND))[2]
    print(up, pres.vec[0])
    deflist1.append(up)
    preslist1.append(pres.vec[0])
    scenep.Redraw()
    if err > 0.05: break

In [ ]:
loadsteps2 = np.linspace(1,0,41,endpoint=True)
deflist2 = []
preslist2 = []
for l in loadsteps2[:]:
    loadpar.Set(l)
    print(f"loadfactor l = {l}")

    # ComputeAveredNV(averednv)
    #u.vec.data += 0.2*uinit.vec
    with TaskManager():
        err, nit = NewtonWithLinesearch(bfA, q.vec, maxnewton=20, factor=1.2,scene=scenep)
    if err > 0.05:
        u.vec.data -= uinit.vec
        err, nit = NewtonWithLinesearch(bfA, q.vec, maxnewton=50, factor=1.2,scene=scenep, damp=0.5)




    up = u(mesh(0,0,m_top-radius_top,BBND))[2]
    print(up, pres.vec[0])
    deflist2.append(up)
    preslist2.append(pres.vec[0])
    scenep.Redraw()
    if err > 0.05: break

In [ ]:
preslist_infl_2d = [-1.8961848599852084e-25, 3.3983754853498845e-07, 7.763942847455647e-07, 1.330919995989358e-06, 1.9962238456610813e-06, 2.7540144140972858e-06, 3.5821350870161346e-06, 4.456865612537901e-06, 5.353982474022833e-06, 6.248569542826046e-06, 7.1160139252451565e-06, 7.9315172352789e-06, 8.670369643139943e-06, 9.307790332068749e-06, 9.818529901197482e-06, 1.0175888058533135e-05, 1.034944669773048e-05, 1.0299631474951684e-05, 9.962536898867782e-06, 9.189584039371606e-06, 1.8916833308090392e-06, 1.937547167281082e-06, 2.083258264644795e-06, 2.3193437738758658e-06, 2.640517177057687e-06, 3.0451657299595695e-06, 3.534075329679005e-06, 4.110611704883104e-06, 4.778450393401269e-06, 5.5424936922297165e-06, 6.408282063348965e-06, 7.381864670741366e-06, 8.46971888348869e-06, 9.678700890776759e-06, 1.101601639949693e-05, 1.2489204981736619e-05, 1.4106134245912215e-05, 1.5875001443049723e-05, 1.780434087505267e-05, 1.9903035834119654e-05, 2.2180333949033775e-05]
loadsteps_infl_2d = np.array([0.   , 0.025, 0.05 , 0.075, 0.1  , 0.125, 0.15 , 0.175, 0.2  ,
       0.225, 0.25 , 0.275, 0.3  , 0.325, 0.35 , 0.375, 0.4  , 0.425,
       0.45 , 0.475, 0.5  , 0.525, 0.55 , 0.575, 0.6  , 0.625, 0.65 ,
       0.675, 0.7  , 0.725, 0.75 , 0.775, 0.8  , 0.825, 0.85 , 0.875,
       0.9  , 0.925, 0.95 , 0.975, 1.   ])
deflist_infl_2d = [-7.14743362769988e-20, 0.3192712807036829, 0.6200846700729091, 0.8980187961750709, 1.1561103162298603, 1.3984115253919402, 1.6280812084624967, 1.8473242447496458, 2.057536890901596, 2.259731301269082, 2.454716117321276, 2.643139004884663, 2.825715931852249, 3.00329852463527, 3.1769439390457435, 3.347978558032429, 3.5180983651465287, 3.6896111482669913, 3.866158868112432, 4.055713811123425, 7.433510837510213, 7.6259022744963225, 7.823373688260419, 8.024885122134826, 8.22901922263942, 8.434552731651051, 8.6405927966084, 8.846625089257325, 9.052321878506426, 9.25755849768301, 9.462328845897021, 9.666702369724407, 9.870793317189401, 10.0747391143911, 10.27868522323186, 10.482774563615507, 10.687140192484987, 10.891900371940602, 11.09715544767304, 11.30298613392421, 11.509452899251453]
preslist_defl_2d = [2.218033767646993e-05, 1.990303655319609e-05, 1.7804341975870687e-05, 1.5875003055971616e-05, 1.4106136549707292e-05, 1.2489208226682457e-05, 1.101602094174996e-05, 9.678707247844131e-06, 8.469727825865545e-06, 7.381877377097522e-06, 6.408300390065131e-06, 5.5425206540491165e-06, 4.778491041713937e-06, 4.110674762044388e-06, 3.5341773527657987e-06, 3.0449620378297576e-06, 2.640201091506893e-06, 2.3193438459145958e-06, 2.083258484163715e-06, 1.93754776797421e-06, 1.8916815012789116e-06, 1.957226928264081e-06, 2.140633682108756e-06, 2.4337199845329313e-06, 2.812790556971387e-06, 3.2504331974309507e-06, 3.730301330575946e-06, 4.2617344086950715e-06, 4.924653931573894e-06, 7.931517227099118e-06, 7.116013927075168e-06, 6.2485695436985616e-06, 5.353939977422523e-06, 4.456958483435207e-06, 3.5821154688761224e-06, 2.7539150653728325e-06, 1.9962238553573178e-06, 1.3309200146594054e-06, 7.763942908680207e-07, 3.3983753665375825e-07, -5.443762638883722e-16]
loadsteps_defl_2d = np.array([1.   , 0.975, 0.95 , 0.925, 0.9  , 0.875, 0.85 , 0.825, 0.8  ,
       0.775, 0.75 , 0.725, 0.7  , 0.675, 0.65 , 0.625, 0.6  , 0.575,
       0.55 , 0.525, 0.5  , 0.475, 0.45 , 0.425, 0.4  , 0.375, 0.35 ,
       0.325, 0.3  , 0.275, 0.25 , 0.225, 0.2  , 0.175, 0.15 , 0.125,
       0.1  , 0.075, 0.05 , 0.025, 0.   ])
deflist_defl_2d = [11.509452504368207, 11.302986195155405, 11.097155540040974, 10.891900501971254, 10.687140368517184, 10.482774797269876, 10.278685531760253, 10.074739524498845, 9.870793871220856, 9.666703135788532, 9.462329934788263, 9.257560091820334, 9.052324282689113, 8.84662882215057, 8.640598822705295, 8.434536526886179, 8.228993812358631, 8.024885119567907, 7.823373678918668, 7.625902238346803, 7.433506754187233, 7.245610642456996, 7.0585555637959265, 6.865483222655151, 6.6583516828249145, 6.429777061361523, 6.172268241438343, 5.874038188151247, 5.503754445888128, 2.6431389930615605, 2.454716117477863, 2.2597313013435247, 2.0575101836707077, 1.8473234022143497, 1.628111720332751, 1.3984464254550217, 1.156110312714125, 0.8980187892255623, 0.6200846676776269, 0.319271286157357, 9.00982587333133e-10]


plt.plot(preslist1, loadsteps1, "xr", label="inflating shell")
plt.plot(preslist_infl_2d, loadsteps_infl_2d, "xb", label="inflating axisym")
plt.xlabel("internal pressure")
plt.ylabel("mean normal displacement")
plt.legend()

In [ ]:
plt.plot(preslist1, loadsteps1, "xr", label="inflating shell")
plt.plot(preslist2, loadsteps2, "+r", label="inflating shell")
plt.plot(preslist_infl_2d, loadsteps_infl_2d, "xb", label="inflating axisym")
plt.plot(preslist_defl_2d, loadsteps_defl_2d, "+b", label="deflating axisym")
plt.xlabel("internal pressure")
plt.ylabel("mean normal displacement")
plt.legend()

In [ ]:
plt.plot(preslist1, deflist1, "xr", label="inflating shell")
plt.plot(preslist2, deflist2, "+r", label="inflating shell")
plt.plot(preslist_infl_2d, deflist_infl_2d, "xb", label="inflating axisym")
plt.plot(preslist_defl_2d, deflist_defl_2d, "+b", label="deflating axisym")
plt.xlabel("internal pressure")
plt.ylabel("mean normal displacement")
plt.legend()